In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Image Retrieval'
!ls

/content/drive/MyDrive/Image Retrieval
 augmentation.ipynb
 binary_scenario
 binary_scenario_augmented_400X
 checkpoint
 citra_split.ipynb
 comparing_graph.ipynb
 comparing_retrieval.ipynb
 core
'data binary splitting.ipynb'
'data subclass splitting.ipynb'
 extract_feature_binary.ipynb
 extract_feature_subclass.ipynb
 original
 paper
 reconstruction
 retrieval_1_100X.ipynb
 retrieval_1_100X_subclass.ipynb
 retrieval_1_200X.ipynb
 retrieval_1_200X_subclass.ipynb
 retrieval_1_400X.ipynb
 retrieval_1_400X_subclass.ipynb
 retrieval_1_40X_subclass.ipynb
 subclass_400
 subclass_scenario
 train_auto_encoder_magnification_1_100.ipynb
 train_auto_encoder_magnification_1_200.ipynb
 train_auto_encoder_magnification_1_400.ipynb
 train_auto_encoder_magnification_1_40.ipynb
 train_auto_encoder_subclass_1_100.ipynb
 train_auto_encoder_subclass_1_200.ipynb
 train_auto_encoder_subclass_1_400.ipynb
 train_auto_encoder_subclass_1_40.ipynb
 training_1_100
 training_1_100.h5
 training_1_100.json
 training_

In [3]:
import numpy as np
import json
import os
import cv2
from core.AutoEncoder1 import ConvAutoEncoder
from tensorflow.keras.models import Model

In [4]:
base_dataset = "subclass_scenario"
class_dir = ['tubular_adenoma', 'phyllodes_tumor', 'papillary_carcinoma',
             'mucinous_carcinoma', 'lobular_carcinoma', 'fibroadenoma',
             'ductal_carcinoma', 'adenosis'
             ]
magnification = '40X'
IMAGE_SIZE = (256, 256)

In [5]:
print("[INFO] indexing file images BreaKHis dataset...")
# indexing file images
dataset_train = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'train', magnification, class_item)
    for file in os.listdir(cur_dir):
        dataset_train.append(os.path.join(cur_dir, file))

[INFO] indexing file images BreaKHis dataset...


In [6]:
print("train:", len(dataset_train))

train: 1632


In [7]:
print("[INFO] load images BreaKHis dataset...")
#  load images
train_images = []
for image_path in dataset_train:
    if ".png" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        train_images.append(image)

[INFO] load images BreaKHis dataset...


In [8]:
# normalization
print("[INFO] normalization...")
train_x = np.array(train_images).astype("float32") / 255.0

[INFO] normalization...


In [9]:
auto_encoder = ConvAutoEncoder.build(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
# load our auto_encoder from disk
print("[INFO] loading auto encoder model...")
auto_encoder.load_weights("training_1_40_subclass/cp.ckpt")

[INFO] loading auto encoder model...


In [10]:
# create the encoder model which consists of *just* the encoder
# portion of the auto encoder
encoder = Model(inputs=auto_encoder.input,
	outputs=auto_encoder.get_layer("encoded").output)

# quantify the contents of our input images using the encoder
print("[INFO] encoding images...")
features = encoder.predict(train_x)

[INFO] encoding images...


In [11]:
indexes = list(range(0, train_x.shape[0]))
features_array = [[float(x) for x in y] for y in features]
labels = [path.split("/")[3] for path in dataset_train]
data = {"indexes": indexes, "features": features_array, "locations": dataset_train, "labels":labels}

In [12]:
with open('training_1_indexed_40_subclass.json', 'w') as f:
    json.dump(data, f)